In [ ]:
import numpy as np
import cv2
import datetime
# import PyCapture2 as fc2
import PySpin
import sys
import time
import os
import re
import skimage.io as io
import msvcrt

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import serial
import re
import csv
#import seaborn as sns
import warnings

import glob
import msvcrt
import winsound
import sklearn
import time

import time
import os

import shutil
import sys
from datetime import datetime

import threading
import queue



# packages for calibration
from sklearn.cluster import KMeans
import peakutils

import importlib

import pySpinCamSetup
import pySpinCamSetup as camS

In [ ]:
_ = importlib.reload(pySpinCamSetup)

In [ ]:
camS.comConnect()

In [ ]:
# check number of cameras
system = PySpin.System.GetInstance()

# Retrieve list of cameras from the system
cam_list = system.GetCameras()
numCams = cam_list.GetSize()


print("Number of cameras detected: ", numCams)

if not numCams:
    #raise ValueError("Insufficient number of cameras. Exiting...")
    print("Insufficient number of cameras. Exiting...")
    exit()


In [ ]:
system = PySpin.System.GetInstance()

# Retrieve list of cameras from the system
cam_list = system.GetCameras()
num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)

In [ ]:
for i, cam in enumerate(cam_list):

    # Retrieve TL device nodemap
    nodemap_tldevice = cam.GetTLDeviceNodeMap()

for i, cam in enumerate(cam_list):

    # Initialize camera
    cam.Init()


In [ ]:
for i, cam in enumerate(cam_list):
    node_acquisition_mode = PySpin.CEnumerationPtr(cam.GetNodeMap().GetNode('AcquisitionMode'))

In [ ]:
def acquire_images(cam_list):
    """
    This function acquires and saves 10 images from each device.

    :param cam_list: List of cameras
    :type cam_list: CameraList
    :return: True if successful, False otherwise.
    :rtype: bool
    """

    print('*** IMAGE ACQUISITION ***\n')

    result = True

    # Prepare each camera to acquire images
    #
    # *** NOTES ***
    # For pseudo-simultaneous streaming, each camera is prepared as if it
    # were just one, but in a loop. Notice that cameras are selected with
    # an index. We demonstrate pseduo-simultaneous streaming because true
    # simultaneous streaming would require multiple process or threads,
    # which is too complex for an example. 
    #
    # Serial numbers are the only persistent objects we gather in this
    # example, which is why a vector is created.
    for i, cam in enumerate(cam_list):

        # Set acquisition mode to continuous
        node_acquisition_mode = PySpin.CEnumerationPtr(cam.GetNodeMap().GetNode('AcquisitionMode'))
        if not PySpin.IsAvailable(node_acquisition_mode) or not PySpin.IsWritable(node_acquisition_mode):
            print('Unable to set acquisition mode to continuous (node retrieval; camera %d). Aborting... \n' % i)
            return False

        node_acquisition_mode_continuous = node_acquisition_mode.GetEntryByName('Continuous')
        if not PySpin.IsAvailable(node_acquisition_mode_continuous) or not PySpin.IsReadable(
                node_acquisition_mode_continuous):
            print('Unable to set acquisition mode to continuous (entry \'continuous\' retrieval %d). \
            Aborting... \n' % i)
            return False

        acquisition_mode_continuous = node_acquisition_mode_continuous.GetValue()

        node_acquisition_mode.SetIntValue(acquisition_mode_continuous)

        print('Camera %d acquisition mode set to continuous...' % i)

        # Begin acquiring images
        cam.BeginAcquisition()

        print('Camera %d started acquiring images...' % i)

        # Retrieve device serial number for filename0
        node_device_serial_number = PySpin.CStringPtr(cam.GetTLDeviceNodeMap().GetNode('DeviceSerialNumber'))

        if PySpin.IsAvailable(node_device_serial_number) and PySpin.IsReadable(node_device_serial_number):
            device_serial_number = node_device_serial_number.GetValue()
            print('Camera %d serial number set to %s...' % (i, device_serial_number))

        print()

    # Retrieve, convert, and save images for each camera
    #
    # *** NOTES ***
    # In order to work with simultaneous camera streams, nested loops are
    # needed. It is important that the inner loop be the one iterating
    # through the cameras; otherwise, all images will be grabbed from a
    # single camera before grabbing any images from another.
    n = 1
    for i, cam in enumerate(cam_list):
        try:
            # Retrieve next received image and ensure image completion
            image_result = cam.GetNextImage()

            if image_result.IsIncomplete():
                print('Image incomplete with image status %d ... \n' % image_result.GetImageStatus())
            else:
                # Print image information
                width = image_result.GetWidth()
                height = image_result.GetHeight()
                print('Camera %d grabbed image, width = %d, height = %d' % (i, width, height))

                # Convert image to mono 8
                image_converted = image_result.Convert(PySpin.PixelFormat_Mono8, PySpin.HQ_LINEAR)

                # Create a unique filename
                if device_serial_number:
                    filename = 'AcquisitionMultipleCamera-%s-%d-%i.jpg' % (device_serial_number, n, i)
                else:
                    filename = 'AcquisitionMultipleCamera-%d-%d-%i.jpg' % (i, n)

                # Save image
                image_converted.Save(filename)
                print('Image saved at %s' % filename)

            # Release image
            image_result.Release()
            print()

        except PySpin.SpinnakerException as ex:
            print('Error: %s' % ex)
            result = False

    # End acquisition for each camera
    #
    # *** NOTES ***
    # Notice that what is usually a one-step process is now two steps
    # because of the additional step of selecting the camera. It is worth
    # repeating that camera selection needs to be done once per loop.
    #
    # It is possible to interact with cameras through the camera list with
    # GetByIndex(); this is an alternative to retrieving cameras as
    # CameraPtr objects that can be quick and easy for small tasks.
    for cam in cam_list:

        # End acquisition
        cam.EndAcquisition()


    return result


In [ ]:
for cam in cam_list:

    # End acquisition
    cam.EndAcquisition()

In [ ]:
acquire_images(cam_list)
    

In [ ]:
for i, cam in enumerate(cam_list):

            image_result = cam.GetNextImage()

In [ ]:
for i, cam in enumerate(cam_list):
    print(i)
    print(cam)

In [ ]:
import AcquisitionMultipleCamera

In [ ]:
import importlib

_ = importlib.reload(AcquisitionMultipleCamera)

In [ ]:
AcquisitionMultipleCamera.main()

In [ ]:
# Retrieve singleton reference to system object
system = PySpin.System.GetInstance()

# Get current library version
version = system.GetLibraryVersion()
print('Library version: %d.%d.%d.%d' % (version.major, version.minor, version.type, version.build))

# Retrieve list of cameras from the system
cam_list = system.GetCameras()

num_cameras = cam_list.GetSize()

print('Number of cameras detected: %d' % num_cameras)

In [ ]:
node_acquisition_mode = PySpin.CEnumerationPtr(cam.GetNodeMap().GetNode('AcquisitionMode'))

In [ ]:
# Get system
system = PySpin.System.GetInstance()

# Get camera list
cam_list = system.GetCameras()

# Figure out which is primary and secondary (usually webcam is primary and Flea3 is secondary)
cam = cam_list.GetByIndex(0)

# Initialize camera
cam.Init()

In [ ]:
# Retrieve TL device nodemap and print device information
nodemap_tldevice = cam.GetTLDeviceNodeMap()


# Retrieve GenICam nodemap
nodemap = cam.GetNodeMap()

In [ ]:
del cam

# Clear camera list before releasing system
cam_list.Clear()

# Release system instance
system.ReleaseInstance()


In [ ]:
def print_device_info(nodemap):
    """
    This function prints the device information of the camera from the transport
    layer; please see NodeMapInfo example for more in-depth comments on printing
    device information from the nodemap.

    :param nodemap: Transport layer device nodemap from camera.
    :type nodemap: INodeMap
    :return: True if successful, False otherwise.
    :rtype: bool
    """
    try:
        result = True
        print('\n*** DEVICE INFORMATION ***\n')

        # Retrieve and display Device Information
        node_device_information = PySpin.CCategoryPtr(nodemap.GetNode('DeviceInformation'))
        if PySpin.IsAvailable(node_device_information) and PySpin.IsReadable(node_device_information):
            features = node_device_information.GetFeatures()
            for feature in features:
                node_feature = PySpin.CValuePtr(feature)
                print('%s: %s' % (node_feature.GetName(),
                                  node_feature.ToString() if PySpin.IsReadable(node_feature) else 'Node not readable'))

        else:
            print('Device control information not available.')

    except PySpin.SpinnakerException as ex:
        print('Error: %s' % ex)
        result = False

    return result


In [ ]:
print_device_info(nodemap)

In [ ]:
 # Get system
system = PySpin.System.GetInstance()

# Get camera list
cam_list = system.GetCameras()

# Figure out which is primary and secondary (usually webcam is primary and Flea3 is secondary)
cam = cam_list.GetByIndex(0)

# Initialize camera
cam.Init()

In [ ]:
trigger_mode = PySpin.CEnumerationPtr(nodemap.GetNode('TriggerMode'))

In [ ]:
trigger_mode.GetDescription()

In [ ]:
# Start acquisition
cam.BeginAcquisition()


# Acquire images
image_primary = cam.GetNextImage()
width = image_primary.GetWidth()
height = image_primary.GetHeight()
print("width: " + str(width) + ", height: " + str(height))

# Pixel array
image_array = image_primary.GetData()

In [ ]:
image_primary = cam.GetNextImage()
img = image_primary.GetNDArray()

In [ ]:
img

In [ ]:
io.imshow(img)

In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Stop acquisition
cam.EndAcquisition()

# De-initialize
cam.DeInit()

# Clear references to images and cameras
del image_primary
del cam
del cam_list